## Implementation of Visualizing Data Using t-SNE

#### Aasha Reddy and Madeleine Beckner

#### Due: April 27, 2021

### Abstract

250 words or less. Identify 4-6 key phrases.

t-SNE. In this report, we will implement the t-SNE algorithm as described in the paper Visualizing Data using t-SNE (Hinton and van der Maaten, 2008). We will demonstrate its accuracy and performance through various applications to both real and simulated datasets, optimize the code, and compare this algorithm with two other competing algorithms that address a similar problem.

Key phrases: t-SNE, stochastic neighbor embedding, dimensionality reduction, visualization

### Background

State the research paper you are using. Describe the concept of the algorithm and why it is interesting and/or useful. If appropriate, describe the mathematical basis of the algorithm. Some potential topics for the background include:
What problem does it address?
What are known and possible applications of the algorithm?
What are its advantages and disadvantages relative to other algorithms?
How will you use it in your research?

In this paper, we will implement the t-SNE algorithm for visualizing high-dimensional data as described in the paper Visualizing Data using t-SNE (Hinton and van der Maaten, 2008). This algorithm gives each datapoint a corresponding location in a two or three-dimensional mapping. The t-SNE algorithm improves upon the Stochastic Neighbor Embedding algorithm (Hinton and Roweis, 2002) by reducing issues with points crowding in the center of the map.

Visualizing high-dimensional data in an effective and interpretable way is important to many different areas of application. High-dimensional datasets are common in numerous fields of study, such as biology, chemistry, political science, economics, astronomy, physics, and more. As availability of computing resources continues to increase, so does the ability to collect and store more and more complex and high-dimensional datasets. Effective visualization  presents an interesting and important challenge in analyzing and understanding these complex datasets.

May different high-dimensional data methods have been proposed in recent years, but many of these methods come with certain issues. For example, techniques that use iconographic displays such as pixel-based techniques or Chernoff faces provide tools to visualize high-dimensional data while leaving the interpretation to the viewer. When scaled to thousands of dimensions, as is often the case with real-world data, these methods break down due to human inability to interpret this kind of visual dimension scale. To avoid this issue, methods with dimensionality-reduction have been proposed that convert high-dimensional data into two or three dimensions to be plotted. Popular linear dimensionality reduction methods include PCA, or Principal Components Analysis, and MDS, Multi-Dimensional Scaling. However, issues arise in applications where low-dimensional representations of similar points must be kept close together, because these methods prioritize keeping low-dimensional representations of dissimilar points far apart. To avoid this concern and preserve local data structures, non-linear dimensionality reduction methods have been proposed as well, one of which, Stochastic Neighbor Embedding, forms the basis of this paper. While these types of methods improve over other mentioned concerns and generally perform well on artificial datasets, they tend to break down when introduced to real-world high dimensional datasets because they are not capable of retaining both local and the global structure of the data in a single map. The method t-SNE presented in this paper provides a solution by being capable of capturing both local and global structures effectively. t-SNE has many advantages relative to other algorithms as discussed, and forms a valuable tool in effectively visualizing high-dimensional data. t-SNE has been shown to be better than existing techniques at creating a single map that reveals structure at many different scales.

While t-SNE makes vast improvements in comparison to other existing methods, one should note that this algorithm has its own weaknesses as well. First, it is not clear how t-SNE performs on general dimensionality reduction tasks where d > 3 due to the heavy tails of the Student-t distribution. Additionally, the relatively local nature of t-SNE makes it sensitive to the curse of the intrinsic dimensionality of the data, which means t-SNE might be less successful if it is applied on data sets with a very high intrinsic dimensionality. The most significant potential weakness is that t-SNE is not guaranteed to converge to a global optimum of its cost function due to the non-convexity of the cost function. Therefore, we have to choose several optimization parameters, and the solutions may differ depending on the choice of these parameters. In spite of these drawbacks, t-SNE still outperforms other similar methods and improves upon common issues in high-dimensional visualization methods. In our paper, we will implement and evaluate the t-SNE metric on multiple datasets. 


### Description of Algorithm

First, explain in plain English what the algorithm does. Then describes the details of the algorithm, using mathematical equations or pseudocode as appropriate.

t-SNE is a variation of Stochastic Neighbor Embedding algorithm (Hinton and Roweis, 2002) that simplifies optimization and prevents crowding in the center of the map (also referred to as the crowding problem). In order to understand t-SNE, we must first discuss Stochastic Neighbor Embedding (SNE), which forms a basis for the t-SNE algorithm.

#### Stochastic Neighbor Embedding

SNE first converts Euclidean distances in high dimensions between the data into conditional probabilities. The conditional probability as shown below represents a similarity between the two datapoints. In other words, $p_{j \mid i}$ is the probability that datapoint $x_i$ would choose $x_j$ as its closest point if closest points were chosen in proportion to their probability density under a Normal distribution centered at $x_i$. Therefore, when datapoints are close, the conditional probability will be low (and vice versa).

The conditional probability can be written mathematically as follows:
$$p_{j \mid i} = \frac{\text{exp}(-\lVert x_i - x_j \rVert^2 / 2\sigma_i^2)}{\sum_{k\neq i} \text{exp}(-\lVert x_i - x_k \rVert^2 / 2\sigma_i^2) }$$

The value of $\sigma_i$ is tricky because it must be determined by binary search with a fixed perplexity defined by the user, typically between 5 and 50.

$$\text{Perplexity} = 2^{- \sum_j p_{j \mid i} \text{log}_2 p_{j \mid i}}$$

Next in SNE, we calculate a similar probability for low-dimensional counterparts of the high-dimensional points $x_i$ and $x_j$. This time the variance is set to $\frac{1}{\sqrt2}$:

$$q_{j \mid i} = \frac{\text{exp}(-\lVert y_i - y_j \rVert^2 )}{\sum_{k\neq i} \text{exp}(-\lVert y_i - y_k \rVert^2) }$$ 

These two conditional probabilities will be equal if the map correctly models the similarity between datapoints in high dimensions and low dimensions. Therefore, SNE works by minimizing the difference between $q_{j \mid i}$ and $p_{j \mid i}$ in finding the value for $q_{j \mid i}$, the low dimensional counterpoint. This difference is minimized in SNE using gradient descent with a KL divergence:

$$ \text{Cost for SNE} = \sum_i \sum_j p_{j \mid i} \text{log} \frac{p_{j \mid i}}{q_{j \mid i}} $$

This cost function is large for using far datapoints to represent near ones, but it is relatively small when using close datapoints to represent far ones. In this way, SNE uses an asymmetrical cost function to preserve local map structure.

The minimization of the cost above is performed with gradient descent. The gradient is defined here:

$$\frac{\delta C}{\delta y_i} = 2 \sum_j (p_{j \mid i} -  q_{j \mid i} + p_{i \mid j} - q_{i \mid j})(y_i - y_j)$$

#### t-SNE and Comparison

t-SNE is very similar to SNE, but differs in two main ways– it uses a different (symmetric) cost function, and a Student-t distribution to compute similarity in low-dimension. The heavy tail of the Student t-distribution alleviates the crowding problem and difficult optimization from the SNE algorithm.

We define the probabilities differently to use the symmetric cost function. We will be using joint probabilities as follows (due to the properties of the Student t-distribution as well as symmetric cost function):


$$q_{ij} = \frac{(1 + \lVert y_i - y_j \rVert^2 )^{-1}}{\sum_{k\neq l} (1 + \lVert y_k - y_l \rVert^2)^{-1} }$$ 

$$p_{ij} = \frac{p_{j \mid i} + p_{i \mid j}}{2n}$$ 

The Perplexity is defined the same as in SNE: $\text{Perplexity} = 2^{- \sum_j p_{j \mid i} \text{log}_2 p_{j \mid i}}$


t-SNE differs from SNE in choice of cost function. The cost function for t-SNE is symmetric and has simpler gradients:

$$ \text{Cost for t-SNE} = \sum_i \sum_j p_{ij} \text{log} \frac{p_{ij}}{q_{ij}} $$


The minimization of the cost above is performed with gradient descent. The gradient is defined here (simpler than that of SNE):

$$\frac{\delta C}{\delta y_i} = 4 \sum_j (p_{ij} -  q_{ij})(y_i - y_j) (1 + \lVert y_i - y_j \rVert^2)^{-1}$$




### Describe Optimization for Performance

First implement the algorithm using plain Python in a straightforward way from the description of the algorithm. Then profile and optimize it using one or more appropriate methods, such as:

1. Use of better algorithms or data structures
2. Use of vectorization
3. JIT or AOT compilation of critical functions
4. Re-writing critical functions in C++ and using pybind11 to wrap them
5. Making use of parallelism or concurrency
6. Making use of distributed compuitng

Document the improvement in performance with the optimizations performed.

### Applications to Simulated Datasets

Are there specific inputs that give known outputs (e.g. there might be closed form solutions for special input cases)? How does the algorithm perform on these? 

If no such input cases are available (or in addition to such input cases), how does the algorithm perform on simulated data sets for which you know the "truth"? 

### Applications to Real Datasets

Test the algorithm on the real-world examples in the original paper if possible. Try to find at least one other real-world data set not in the original paper and test it on that. Describe and interpret the results.


#### MNIST dataset

We tested our original algorithm on the MNIST dataset, .
Here is the plot run on the paper author's code implementation, provided on his website.
![img](Report_Plots/paper_MNIST_plot.png)

Here is the plot run on our implementation:

![img2](Report_Plots/our_MNIST_plot.png)

#### IRIS dataset (small dataset for original testing)

### Comparative Analysis to Competing Algorithms

Find two other algorithm that address a similar problem. Perform a comparison - for example, of accuracy or speed. You can use native libraries of the other algorithms - you do not need to code them yourself. Comment on your observations. 

#### Comparison: PCA

##### Speed and Accuracy



### Discussion and Conclusion

Your thoughts on the algorithm. Does it fulfill a particular need? How could it be generalized to other problem domains? What are its limitations and how could it be improved further?

### References

1. van der Maaten, Laurens and Hinton, Geoffrey. "Visualizing Data using t-SNE ." Journal of Machine Learning Research 9 (2008): 2579--2605.
2. Sci-kit Learn implementation: https://github.com/scikit-learn/scikit-learn/blob/95119c13af77c76e150b753485c662b7c52a41a2/sklearn/manifold/_utils.pyx
http://www.sci.utah.edu/~beiwang/publications/Vis_HD_STAR_BeiWang_2015.pdf

### Code